In [1]:
!pip install stanza

     |████████████████████████████████| 432 kB 5.6 MB/s 
     |████████████████████████████████| 168 kB 57.3 MB/s 
  Created wheel for emoji: filename=emoji-1.6.0-py3-none-any.whl size=168256 sha256=d9bec6551d4ac950a0982157d3944b63b0d003896041607230be4380185739af
  Stored in directory: /root/.cache/pip/wheels/f7/d7/74/c720aaf345a042b0c2d74361873258c5e8649b7f11b2ccce49
Successfully built emoji


In [2]:
import stanza
stanza.download('ru')
nlp = stanza.Pipeline('ru')

2021-10-10 01:15:15 INFO: Downloading default packages for language: ru (Russian)...


2021-10-10 01:15:40 INFO: Finished downloading models and saved to /root/stanza_resources.
2021-10-10 01:15:40 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |
| lemma     | syntagrus |
| depparse  | syntagrus |
| ner       | wikiner   |

2021-10-10 01:15:40 INFO: Use device: cpu
2021-10-10 01:15:40 INFO: Loading: tokenize
2021-10-10 01:15:40 INFO: Loading: pos
2021-10-10 01:15:40 INFO: Loading: lemma
2021-10-10 01:15:41 INFO: Loading: depparse
2021-10-10 01:15:41 INFO: Loading: ner
2021-10-10 01:15:43 INFO: Done loading processors!


In [3]:
import pandas as pd

In [4]:
import requests

In [10]:
from tqdm import tqdm

In [5]:
link = "http://nlp.isa.ru/framebank_parser/data/preprocessed_framebank/annotated_corpus.json"
data = requests.get(link).json()

In [6]:
len(data)

59861

In [7]:
def synt_annotate(sent_text):
  doc = nlp(sent_text)
  annotated_list = []
  for sent in doc.sentences:
    for word in sent.words:
      word_info = {'id': word.id, 'text': word.text, 'upos': word.upos, 'head': word.head, 'deprel': word.deprel}
      if word.feats:
        word_info['feats'] = word.feats
      else:
        word_info['feats'] = '_'
      annotated_list.append(word_info)
  return annotated_list

In [8]:
def join_annotations(synt_annotated, json_sentence):
  for i, word in enumerate(synt_annotated):
    json_sentence[i]['id'] = word['id']
    json_sentence[i]['upos'] = word['upos']
    json_sentence[i]['head'] = word['head']
    json_sentence[i]['deprel'] = word['deprel']
    json_sentence[i]['feats'] = word['feats']
    if word['upos'] == 'PUNCT':
      json_sentence[i]['lemma'] = word['text']
      json_sentence[i]['feat'] = '_'
      json_sentence[i]['sem'] = '_'
      json_sentence[i]['sem2'] = '_'
    if 'fillpred' not in json_sentence[i]:
      json_sentence[i]['fillpred'] = '_'
    if 'rolepred1' not in json_sentence[i]:
      json_sentence[i]['rolepred1'] = '_'
    if 'rank' not in json_sentence[i]:
      json_sentence[i]['rank'] = '_'

In [9]:
def example_annotation(example):
  for sentence in example:
    for word in sentence:
      if word['form'] == '--':
        word['form'] = '—'
    sent_text = ' '.join([word['form'] for word in sentence])
    synt_annotated = synt_annotate(sent_text)
    join_annotations(synt_annotated, sentence)
  return example

In [ ]:
example_annotation(example)

In [11]:
link2 = 'https://raw.githubusercontent.com/olesar/framebank/master/framebank_anno_ex_items.txt'
data2 = pd.read_csv(link2, sep='\t')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
ex_list = []
for item in data2['ExIndex']:
  if item not in ex_list:
    ex_list.append(item)

In [44]:
len(ex_list)

38476

In [13]:
result = {}
for ex_id in tqdm(ex_list[:3000]):
  if str(ex_id) in data:
    try:
      result[str(ex_id)] = example_annotation(data[str(ex_id)])
    except IndexError:
      pass

100%|██████████| 3000/3000 [1:42:31<00:00,  2.05s/it]


In [14]:
len(result)

2858

In [15]:
df = pd.concat([pd.DataFrame(item) for item in result['35300']], ignore_index=True)
for key in result:
  if key != '35300':
    ex_df = pd.concat([pd.DataFrame(item) for item in result[key]], ignore_index=True)
    df = pd.concat([df, ex_df], ignore_index=True)

In [16]:
df

,lemma,sem,feat,sem2,form,id,upos,head,deprel,feats,fillpred,rolepred1,rank,pred
0,Собачкин,t:hum r:propn t:famn,S famn m anim sg nom,,Собачкин,1.0,NOUN,7.0,nsubj,Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing,_,_,_,NaN
1,в,,PR,,в,2.0,ADP,5.0,case,_,_,_,_,NaN
2,гоголевский,dt:famn r:rel der:s dt:hum,A m sg loc plen,,гоголевском,3.0,ADJ,5.0,amod,Case=Loc|Degree=Pos|Gender=Masc|Number=Sing,_,_,_,NaN
3,"""",_,_,_,"""",4.0,PUNCT,5.0,punct,_,_,_,_,NaN
4,отрывок,pc:text r:concr t:text pt:part,S m inan sg loc,pt:residpart r:abstr,Отрывке,5.0,NOUN,7.0,obl,Animacy=Inan|Case=Loc|Gender=Masc|Number=Sing,_,_,_,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277493,всерьез,dt:humq der:a,"ADV,norm",,всерьез,7.0,ADV,10.0,advmod,Degree=Pos,_,_,_,NaN
277494,это,r:dem,"SPRO,gen,n,norm,sg",,этого,8.0,PRON,10.0,obl,Animacy=Inan|Case=Gen|Gender=Neut|Number=Sing,_,_,_,NaN
277495,не,,"PART,norm",,не,9.0,PART,10.0,advmod,_,_,_,_,NaN
277496,принимать,d:pref,"V,act,indic,m,norm,pf,praet,sg",,принял,10.0,VERB,2.0,conj,Aspect=Perf|Gender=Masc|Mood=Ind|Number=Sing|T...,_,_,_,NaN


In [17]:
df['upos2'] = 'PUNCT'
for idx, line in enumerate(df.isna()['feat']):
  if df['feat'].notna()[idx] and df['feat'][idx] != '_':
    df['upos2'][idx] = df['feat'][idx].split()[0]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


Дальше можно создавать любые запросы, которые можно выполнить с подобными данными в датафрейме.

In [20]:
#части речи и части речи
df.loc[df['upos2'] == 'V']['upos'].value_counts()

VERB     33100
AUX       1800
NOUN       157
ADJ        146
PROPN       26
ADV         10
INTJ         4
ADP          2
DET          1
PART         1
Name: upos, dtype: int64

In [21]:
df.loc[(df['upos'] == 'ADP') & (df['upos2'] == 'V')]

,lemma,sem,feat,sem2,form,id,upos,head,deprel,feats,fillpred,rolepred1,rank,pred,upos2
121443,благодарить,t:speech ca:noncaus,V ipf tran ger act praes,,благодаря,4.0,ADP,5.0,case,_,_,_,Предикат,13319.0,V
123318,благодарить,t:speech ca:noncaus,V ipf tran ger act praes,,благодаря,27.0,ADP,28.0,case,_,_,_,Предикат,17180.0,V


In [30]:
df.loc[[121444, 123319]]['form']

121444    Бога
123319    Бога
Name: form, dtype: object

In [18]:
#части речи и роли
df.loc[df['upos'] == 'PRON']['rolepred1'].value_counts()[1:4]

субъект психологического состояния    253
агенс                                 155
причина                               113
Name: rolepred1, dtype: int64

In [34]:
#вершина и части речи
df.loc[df['deprel'] == 'root']['upos'].value_counts()[:5]

VERB     11923
NOUN      1797
PUNCT     1461
ADJ       1402
ADV        583
Name: upos, dtype: int64

In [35]:
df.loc[df['deprel'] == 'root']['upos2'].value_counts()[:5]

V          10823
PUNCT       1842
S           1666
A            899
PRAEDIC      444
Name: upos2, dtype: int64

In [43]:
#вершина и роли
df.loc[df['deprel'] == 'obl']['rolepred1'].value_counts()[:4]

_                 22822
причина             108
контрагент           90
конечная точка       78
Name: rolepred1, dtype: int64

In [39]:
df['upos'].value_counts()[:5]

PUNCT    60045
NOUN     52501
VERB     37361
ADP      23685
ADJ      20439
Name: upos, dtype: int64

In [40]:
df['upos2'].value_counts()[:5]

PUNCT    62247
S        55297
V        35247
PR       22496
CONJ     17713
Name: upos2, dtype: int64

In [41]:
df.loc[df['id'] == 1]

,lemma,sem,feat,sem2,form,id,upos,head,deprel,feats,fillpred,rolepred1,rank,pred,upos2
0,Собачкин,t:hum r:propn t:famn,S famn m anim sg nom,,Собачкин,1.0,NOUN,7.0,nsubj,Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing,_,_,_,NaN,S
58,не,,PART,,не,1.0,PART,2.0,advmod,_,_,_,_,NaN,PART
80,идти,ca:noncaus t:move d:root,V ipf intr act sg 2p imper,ca:noncaus t:move d:root,Иди,1.0,VERB,0.0,root,Aspect=Imp|Mood=Imp|Number=Sing|Person=2|VerbF...,_,_,_,NaN,V
105,ну,,PART,,Ну,1.0,PART,9.0,parataxis,_,_,_,_,NaN,PART
114,спешить,der:v,V ipf intr act pl praes 3p indic,der:v,спешат,1.0,VERB,0.0,root,Aspect=Imp|Mood=Ind|Number=Plur|Person=3|Tense...,_,_,_,NaN,V
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277413,в,,"PR,norm",,В,1.0,ADP,2.0,case,_,_,_,_,NaN,"PR,norm"
277444,чиновник,t:hum r:concr der:s,"S,anim,m,nom,norm,sg",t:hum r:concr ev:neg,Чиновник,1.0,NOUN,2.0,nsubj,Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing,_,_,_,NaN,"S,anim,m,nom,norm,sg"
277464,так,der:a dt:degr r:dem,"ADVPRO,norm",der:apro dt:degr r:dem,Так,1.0,ADV,3.0,advmod,Degree=Pos,_,_,_,NaN,"ADVPRO,norm"
277468,герр,t:hum r:propn r:concr t:famn,"S,anim,m,nom,norm,sg",,Герр,1.0,PROPN,4.0,nsubj,Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing,_,_,_,NaN,"S,anim,m,nom,norm,sg"


In [42]:
df.loc[df['rolepred1'] == 'субъект психологического состояния']['upos'].value_counts()

PRON     253
NOUN     145
PROPN     46
DET       11
ADJ        3
VERB       2
NUM        1
Name: upos, dtype: int64